In [ ]:
import glob
import re

# Third-party
from astropy.io import ascii, fits
import astropy.coordinates as coord
from astropy.table import Table
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

from astropy.constants import c
from scipy.integrate import simps

For info about the model spectra, see Adam's page:
https://www.astro.princeton.edu/~burrows/non/non.html

For info about the filter curves, see the HSC wiki page:
http://hscsurvey.pbworks.com/w/page/78016541/response_functions

In [ ]:
def load_model(model_spec_file, lambda_=False):
    model_spec = ascii.read(model_spec_file)
    
    # Load the model spectrum - wavelength and f_nu are given
    model_wvln = (model_spec['LAMBDA(mic)'] * u.micron).to(u.angstrom)
    model_nu = model_wvln.to(u.Hz, u.spectral())
    model_fnu = (model_spec['FDET(milliJ)'] * u.milliJansky).to(u.erg/u.s/u.cm**2/u.Hz)
    
    if lambda_:
        return model_wvln, model_fnu.to(u.erg/u.s/u.cm**2/u.Angstrom, 
                                        u.spectral_density(model_wvln))
    else:
        return model_nu, model_fnu

def load_filter(filter_file, lambda_=False):
    # Load the filter response curve
    filter_resp = ascii.read(filter_file, names=['wavelength', 'response'])
    
    # response is per photon
    filter_wvln = filter_resp['wavelength'] * u.angstrom
    filter_nu = filter_wvln.to(u.Hz, u.spectral())
    filter_resp = filter_resp['response']
    
    if lambda_:
        return filter_wvln, filter_resp
    else:
        return filter_nu, filter_resp
    
def get_abs_mag(model_spec_file, filter_resp_file):
    model_nu, model_fnu = load_model(model_spec_file)
    
    # Load the filter response curve
    filter_nu, filter_resp = load_filter(filter_resp_file) 
    
    # filter response on the model wavelength (freq) grid
    filter_resp_model = np.interp(model_nu.value, filter_nu.value, filter_resp)
    
    fnu1 = simps((model_fnu * filter_resp_model)[np.argsort(model_nu)], 
                 model_nu[np.argsort(model_nu)]) * model_fnu.unit*u.Hz

    fnu2 = simps(filter_resp_model[np.argsort(model_nu)], 
                 model_nu[np.argsort(model_nu)]) * u.Hz * 3631*u.Jansky

    return -2.5*np.log10(fnu1 / fnu2)

In [ ]:
# test with known files
get_abs_mag("../data/non.hubeny.burrows/c100_neq/T1800_g50_d2f0.c100", 
            "../data/hsc_responses_v1/hsc_r.dat")

### Plot spectrum, filter curves:

In [ ]:
lam, f_lam = load_model("../data/non.hubeny.burrows/c100_neq/T1800_g50_d2f0.c100", 
                        lambda_=True)
_idx = (lam > 3800*u.AA) & (lam < 11000*u.AA)
lam = lam[_idx]
f_lam = f_lam[_idx]

fig,ax = plt.subplots(1, 1, figsize=(10,6))

for filter_fname in glob.glob('../data/hsc_responses_v1/*.dat'):
    filter_name = pattr.search(filter_fname).groups()[0]
    if filter_name.startswith('nb'): continue
    
    filter_lam, filter_resp = load_filter(filter_fname, lambda_=True)
    ax.plot(filter_lam, filter_resp, marker='', color='#555555')

ax.plot(lam, f_lam / f_lam.max(), marker='', drawstyle='steps-mid')

ax.set_xlim(3800, 11000)

Loop over all filters, all model spectra (range of temperatures):

In [ ]:
pattr = re.compile('hsc_([0-9a-zA-Z]+).dat')

In [ ]:
Ts = np.arange(700, 1800+1, 100)
tbl_data = dict()
for filter_fname in glob.glob('../data/hsc_responses_v1/*.dat'):
    filter_name = pattr.search(filter_fname).groups()[0]
    
    tbl_data[filter_name] = []
    for T in Ts:
        model_fname = "../data/non.hubeny.burrows/c100_neq/T{0:d}_g50_d2f2.c100".format(T)
        mag = get_abs_mag(model_fname, filter_fname).value
        tbl_data[filter_name].append(mag)
tbl_data['T'] = Ts

tbl = Table(tbl_data)
tbl['T'].unit = u.Kelvin

In [ ]:
fig,ax = plt.subplots(1, 1, figsize=(6,6))

colors = ['tab:orange', 'tab:red', '#a50f15', 'black']
for fil, color in zip('rizy', colors):
    ax.plot(tbl['T'], tbl[fil] + 5, label='${0}$'.format(fil), 
            marker='', color=color)
    
ax.legend(loc='lower right', fontsize=18)
    
ax.set_ylim(26, 18.5)
ax.set_title('brown dwarfs at 100 pc')

ax.set_xlabel('$T$ [K]')
ax.set_ylabel('$m$ [mag]')

### Maximum distance given limiting magnitude

In [ ]:
m_lim = dict(g=27, r=26.5, i=26., z=25.5, y=24.7)

In [ ]:
fig,ax = plt.subplots(1, 1, figsize=(6,6))

colors = ['tab:orange', '#de2d26', '#a50f15', 'black']
for fil, color in zip('rizy', colors):
    d_lim = 10**((m_lim[fil] - 0.5 - tbl[fil])/5) * 10.
    ax.plot(tbl['T'], d_lim, label='${0}$'.format(fil), 
            marker='', color=color, linewidth=3)
    
ax.legend(loc='upper left', fontsize=18)
    
# ax.set_ylim(26, 18.5)
ax.set_xlim(700, 1800)
ax.set_title('brown dwarfs in HSC')

ax.set_xlabel('$T$ [K]')
ax.set_ylabel(r'$d_{\rm max}$ [pc]')

ax.axvline(1300., zorder=-1, linestyle='--')

ax.text(1310, 1500, 'L→', ha='left', fontsize=14)
ax.text(1290, 1500, '←T', ha='right', fontsize=14)

fig.tight_layout()
fig.set_facecolor('w')
fig.savefig('hsc_bd.png', dpi=300)